In [ ]:
import os, gzip, json
from time import sleep, time
from mpcontribs.client import load_client
from pymatgen import Structure, MPRester
from urllib.request import urlretrieve
from monty.json import MontyDecoder
from itertools import groupby
from operator import itemgetter

In [ ]:
project = '2dmatpedia'
client = load_client('your-api-key-here')
mpr = MPRester()

Create project (once)

In [ ]:
is_public = True
info = {
    'project': project,
    'is_public': is_public,
    'title': '2DMatPedia',
    'long_title': '2D Materials Encyclopedia',
    'owner': 'migueldiascosta@nus.edu.sg',
    'authors': 'M. Dias Costa, F.Y. Ping, Z. Jun',
    'description': ' '.join('''
    We start from the around 80000 inorganic compounds in the Materials Project database. A geometry-based
    algorithm [PRL] was used to identify layered structures among these compounds. Two-dimensional (2D)
    materials were theoretically exfoliated by extracting one cluster in the standard conventional unit cell
    of the layered structures screened in the above steps. A 20 Å vacuum along the c axis was imposed to
    minimize the interactions of image slabs by periodic condition. Structure matcher tools from Pymatgen were
    used to find duplicates of the exfoliated 2D materials. The standard workflow developed by the Materials
    Project was used to perform high-throughput calculations for all the layered bulk and 2D materials screened
    in this project. The calculations were performed by density functional theory as implemented in the Vienna
    Ab Initio Simulation Package (VASP) software with Perdew-Burke-Ernzerhof (PBE) approximation for the
    exchange-correlation functional and the frozen-core all-electron projector-augmented wave (PAW) method for
    the electron-ion interaction. The cutoff energy for the plane wave expansion was set to 520 eV.
    '''.replace('\n', '').split()),
    'urls': {
        'WWW': 'http://www.2dmatpedia.org',
        'PRL': 'https://doi.org/10.1103/PhysRevLett.118.106101'    
    }
}
# client.projects.create_entry(project=info).result()
# client.projects.get_entry(pk=project, _fields=['_all']).result()

Download source data

In [ ]:
config = {
    "file": "http://www.2dmatpedia.org/static/db.json.gz",
    "details": "http://www.2dmatpedia.org/2dmaterials/doc/{}",
    'columns': {
        'material_id': {'name': 'details'},
        'exfoliation_energy_per_atom': {'name': 'Eₓ', 'unit': 'eV'},
        'energy_per_atom': {'name': 'E', 'unit': 'meV'},
        'energy_vdw_per_atom': {'name': 'ΔE|vdW', 'unit': 'meV'},
        'bandgap': {'name': 'ΔE', 'unit': 'meV'},
    }
}

In [ ]:
raw_data = []  # as read from raw files
dbfile = config['file'].rsplit('/')[-1]
if not os.path.exists(dbfile):
    print('downloading', dbfile, '...')
    urlretrieve(config['file'], dbfile)

with gzip.open(dbfile, 'rb') as f:
    for line in f:
        raw_data.append(json.loads(line, cls=MontyDecoder))

len(raw_data)

In [ ]:
# get data dict from raw data entry
def get_data(rd, identifier):
    data = {}
    for k, col in config['columns'].items():
        hdr, unit = col['name'], col.get('unit')
        if k == 'material_id':
            data[hdr] = config[hdr].format(rd[k])
        elif k in rd:
            if unit:
                try:
                    float(rd[k])
                except ValueError:
                    continue
            data[hdr] = f'{rd[k]} {unit}' if unit else rd[k]
    return data

In [ ]:
# clean out
entries = client.contributions.get_entries(
    project=project, _fields=['id', 'identifier'], _limit=200
).result()['data']
for idx, d in enumerate(entries):
    client.contributions.delete_entry(pk=d['id']).result()
    print(idx, d['id'], d['identifier'], 'deleted')

In [ ]:
raw_data_sorted = sorted(raw_data, key=itemgetter('source_id'))
raw_data_grouped = {}

for task_id, objects in groupby(raw_data_sorted, key=itemgetter('source_id')):
    raw_data_grouped[task_id] = list(objects)        

print(len(raw_data_sorted), len(raw_data_grouped))

In [ ]:
count = client.contributions.get_entries(project=project, _limit=1).result()['total_count']

for idx, (task_id, objects) in enumerate(raw_data_grouped.items()):
    if not idx%25:
        if idx > 0:
            stop = time()
            duration = stop-start
            print(duration)
        start = time()
        print(idx, task_id)

    if idx < count: # redo last
        continue

    try:
        identifier = mpr.get_materials_id_from_task_id(task_id)
    except Exception as ex:
        print(idx, task_id, 'invalid')
        continue

        
    contrib = {'project': project, 'is_public': is_public, 'identifier': identifier, 'data': {}}
    structures, names = [], []
    for j, d in enumerate(objects):
        structure = d['structure']
        comp = structure.composition.reduced_formula
        name = f'{comp}|{j}' if comp in names else comp        
        names.append(name)
        structures.append({'label': '2D', 'name': name, 'is_public': is_public})
        structures[-1].update(structure.as_dict())
        label = f'2D|{j}'
        contrib['data'][label] = get_data(d, identifier)
        contrib['data'][label]['formula'] = comp

    ntries = 0
    while ntries < 3:
        try:
            for d in client.contributions.get_entries(
                project=project, identifier=identifier, _fields=['id']
            ).result()['data']:
                client.contributions.delete_entry(pk=d['id']).result()
                print(idx, d['id'], 'deleted')

            cid = client.contributions.create_entry(contribution=contrib).result()['id']
            print(idx, cid, 'created')

            for j, sdct in enumerate(structures):
                sdct['contribution'] = cid
                sid = client.structures.create_entry(structure=sdct).result()['id']
                #print(idx, 'structure', sid, 'created')

            break
#         except Exception as ex:
#             ntries += 1
#             wait = ntries * 30 + (ntries-1) * 30 * 60
#             print('waiting', ntries*30, 'seconds...')
#             sleep(ntries*30)
#             print(idx, identifier, 'continue ...')
    else:
        print('I give up.')
        break
    
    break